# Amplifierd API - Getting Started

This notebook introduces the amplifierd daemon and demonstrates basic API usage.

## What is Amplifierd?

Amplifierd is an HTTP API daemon that provides programmatic access to amplifier configuration management:
- Profile management
- Collection management
- Module discovery and configuration
- Session management for LLM interactions

## Prerequisites

1. Install Python dependencies for running these notebooks:
```bash
source .venv/bin/activate
uv sync # `requests` is the main dependency.
```

2. The amplifierd daemon should be running:
```bash
uv run python -m amplifierd
# Or run from VS Code "Run and Debug" panel.
```

Default configuration:
- Host: `127.0.0.1`
- Port: `8420`
- Base URL: `http://127.0.0.1:8420`

## Setup

Import required libraries and configure the API client:

In [17]:
import json

import requests

# API Configuration
BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


# Helper function for pretty printing JSON responses
def print_response(response: requests.Response) -> None:
    """Print HTTP response with status and formatted JSON."""
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            print(json.dumps(response.json(), indent=2))
        except json.JSONDecodeError:
            print(response.text)


print("✓ Setup complete")
print(f"✓ Base URL: {BASE_URL}")

✓ Setup complete
✓ Base URL: http://127.0.0.1:8420


## Health Check

Verify the daemon is running and responsive:

In [18]:
response = requests.get(f"{API_BASE}/health")
print_response(response)

Status: 200 OK
{
  "status": "healthy"
}


## Status Endpoint

Get detailed daemon status information:

In [19]:
response = requests.get(f"{API_BASE}/status")
print_response(response)

if response.ok:
    status = response.json()
    print(f"\nDaemon has been running for {status['uptimeSeconds']} seconds")

Status: 200 OK
{
  "status": "running",
  "version": "0.1.0",
  "uptimeSeconds": 57.23276948928833,
  "rootDir": "/data/repos/msft/payneio/amplifierd/.amplifierd"
}

Daemon has been running for 57.23276948928833 seconds


## Root Endpoint

Get API information and available endpoints:

In [20]:
response = requests.get(BASE_URL)
print_response(response)

Status: 200 OK
{
  "name": "amplifierd",
  "version": "0.1.0",
  "description": "REST API daemon for amplifier-core",
  "docs": "/docs",
  "openapi": "/openapi.json"
}


## Error Handling

Example of handling API errors:

In [21]:
# Try to access a non-existent endpoint
response = requests.get(f"{API_BASE}/nonexistent")
print_response(response)

# Proper error handling pattern
try:
    response.raise_for_status()
except requests.HTTPError as e:
    print(f"\n❌ Error: {e}")
    if response.status_code == 404:
        print("Resource not found")
    elif response.status_code >= 500:
        print("Server error")

Status: 404 Not Found
{
  "detail": "Not Found"
}

❌ Error: 404 Client Error: Not Found for url: http://127.0.0.1:8420/api/v1/nonexistent
Resource not found


## Testing Connectivity

Create a simple connection test function:

In [22]:
def test_connection() -> bool:
    """Test if amplifierd daemon is accessible."""
    try:
        response = requests.get(f"{API_BASE}/health", timeout=2)
        if response.ok:
            print("✓ Connection successful")
            return True
        print(f"✗ Health check failed: {response.status_code}")
        return False
    except requests.ConnectionError:
        print("✗ Cannot connect to daemon. Is it running?")
        return False
    except requests.Timeout:
        print("✗ Connection timeout")
        return False


test_connection()

✓ Connection successful


True

## Summary

You've learned:
- ✓ How to configure the API client
- ✓ Basic health and status checks
- ✓ Error handling patterns
- ✓ Connection testing

## Next Steps

Continue to the next notebooks:
- **02-sessions-and-messages.ipynb** - Session lifecycle and messaging
- **03-profile-management.ipynb** - Profile operations
- **04-collection-management.ipynb** - Collection management
- **05-module-management.ipynb** - Module discovery and configuration

## Troubleshooting

### Daemon Not Running
```bash
# Start the daemon
python -m amplifierd

# Or with custom port
AMPLIFIER_DAEMON_PORT=8421 python -m amplifierd
```

### Connection Refused
- Check if daemon is running: `ps aux | grep amplifierd`
- Verify port is correct
- Check firewall settings

### Import Errors
```bash
# Install required dependencies
pip install requests
```